<a href="https://colab.research.google.com/github/ENRICOROBLESDELRIO/data_boot_camp_final_project/blob/raquel_branch/First_exploration_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Database set up**



In [ ]:
# Libraries and dependencies

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import math
from sklearn.metrics import accuracy_score
from google.colab import files
from sqlalchemy import create_engine
import psycopg2


# Import for models
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# Google Cloud SQL connexion:
# Code taken from: https://stackoverflow.com/questions/53180140/how-to-access-cloud-sql-from-google-colab
!gcloud auth login
!gcloud config set project endeavor-intelligence-unit
!gcloud sql instances describe finalprojectdb

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=e19kvDhrDf3ar1zw3u72TslBLofvQ8&prompt=consent&access_type=offline&code_challenge=XXd5HnwW7FwK0tEsuerKR4Tl9H0GzJrGEW7TcRuNGXc&code_challenge_method=S256

Enter verification code: 4/1AX4XfWhiOpvldYZBNfRgb4U5vsY9kZd0NB2zplMpRDNWWrYTNexMLEeMrFU

You are now logged in as [racvbs@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].
ERROR: (gcloud.sql.instances.describe) There was no instance 

In [ ]:
# download and initialize the psql proxy
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy

# "connectionName" is from the previous block
!nohup ./cloud_sql_proxy -instances="endeavor-intelligence-unit:northamerica-northeast2:finalprojectdb"=tcp:5432 &
!sleep 30s

--2021-11-13 14:31:59--  https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64
Resolving dl.google.com (dl.google.com)... 173.194.216.91, 173.194.216.93, 173.194.216.136, ...
Connecting to dl.google.com (dl.google.com)|173.194.216.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16638816 (16M) [application/octet-stream]
Saving to: ‘cloud_sql_proxy’

cloud_sql_proxy     100%[===================>]  15.87M  --.-KB/s    in 0.07s   

2021-11-13 14:31:59 (214 MB/s) - ‘cloud_sql_proxy’ saved [16638816/16638816]

nohup: appending output to 'nohup.out'


In [ ]:
#Checking the proxy´s logs 
!cat nohup.out

2021/11/13 14:32:00 current FDs rlimit set to 1048576, wanted limit is 8500. Nothing to do here.
2021/11/13 14:32:01 errors parsing config:
	googleapi: Error 403: The client is not authorized to make this request., notAuthorized


In [ ]:
# Connection to protect access and users to the database: 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Connection to the file with user and access to the database: 

from google.colab import files

with open('/content/drive/MyDrive/Final_project/Second_Deliverable/key_config.txt', "r") as file:
    FileasList = file.readlines()
    username = FileasList[0]
    password = FileasList[1]

    new_line = []
    # Loop through the file
    for i in FileasList:
        # Get rid of any new lines ("enter")
        i = i.replace("\n", "")
        # Add the cleaned data to the new_line list
        new_line.append(i)
    # Set the username to be the first object
    username = new_line[0]
    # Set the password to be the second object
    password = new_line[1]
    # Set the host to be the third object
    host = new_line[2]
    # Set the port to be the four object
    port = new_line[3]
    # Set the database to be the five object
    database = new_line[4]

In [ ]:
# Connexion throught psycopg2 Google Cloud SQL data base

conn = psycopg2.connect(
    host=host, port=port, database=database,
    user=username, password=password)

# **Founders**

## Cleaning



**Libraries**


In [ ]:

# Libraries and dependencies

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import math
from sklearn.metrics import accuracy_score
from google.colab import files

## Import for models
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Query to connect with SQL data base from Founders Demographics 
SQL_Query = pd.read_sql_query('''select * FROM "Founders" ''', conn)
founders_df = pd.DataFrame(SQL_Query)
founders_df.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,"Ciudad de México, México",1,1,0,0,0,0,0,1.0,2020.0,12.0,31.0,32.0,13.0,4,M,0,0,0
1,2,23,"Ciudad de México, México",0,1,0,0,0,0,0,1.0,2014.0,4.0,25.0,32.0,11.0,2,M,0,0,0
2,3,141,México,0,1,0,0,0,0,0,0.0,2015.0,0.0,22.0,28.0,6.0,1,M,0,1,0
3,4,152,"Ciudad de México, México",0,0,0,0,1,0,1,1.0,2017.0,6.0,28.0,32.0,10.0,3,M,1,1,1
4,5,155,México,1,1,0,0,0,0,0,0.0,2015.0,10.0,27.0,33.0,16.0,1,M,1,0,0


In [ ]:
# data types
founders_df.dtypes

id_founder                               int64
id_company                               int64
general_location                        object
manager_exp                              int64
clevel_exp                               int64
engineering_exp                          int64
sales_exp                                int64
finance_exp                              int64
human_capital_exp                        int64
consulting_exp                           int64
founder_exp                            float64
foundation_year                        float64
years_of_experience_as_entreprenuer    float64
age_of_first_entrepreneurship          float64
age                                    float64
years_of_experience                    float64
number_of_startups                       int64
gender                                  object
foreing_studies                          int64
foreing_work_experience                  int64
foreing_founder                          int64
dtype: object

In [ ]:
founders_df.count()

id_founder                             640
id_company                             640
general_location                       629
manager_exp                            640
clevel_exp                             640
engineering_exp                        640
sales_exp                              640
finance_exp                            640
human_capital_exp                      640
consulting_exp                         640
founder_exp                            639
foundation_year                        633
years_of_experience_as_entreprenuer    633
age_of_first_entrepreneurship          566
age                                    634
years_of_experience                    634
number_of_startups                     640
gender                                 640
foreing_studies                        640
foreing_work_experience                640
foreing_founder                        640
dtype: int64

In [ ]:
# Review
founders_df.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,"Ciudad de México, México",1,1,0,0,0,0,0,1.0,2020.0,12.0,31.0,32.0,13.0,4,M,0,0,0
1,2,23,"Ciudad de México, México",0,1,0,0,0,0,0,1.0,2014.0,4.0,25.0,32.0,11.0,2,M,0,0,0
2,3,141,México,0,1,0,0,0,0,0,0.0,2015.0,0.0,22.0,28.0,6.0,1,M,0,1,0
3,4,152,"Ciudad de México, México",0,0,0,0,1,0,1,1.0,2017.0,6.0,28.0,32.0,10.0,3,M,1,1,1
4,5,155,México,1,1,0,0,0,0,0,0.0,2015.0,10.0,27.0,33.0,16.0,1,M,1,0,0


In [ ]:
# Drop the null columns where all values are null
founders_df = founders_df.dropna(axis='columns', how='all')

# Drop the null rows
founders_df = founders_df.dropna()
founders_df.count()

id_founder                             555
id_company                             555
general_location                       555
manager_exp                            555
clevel_exp                             555
engineering_exp                        555
sales_exp                              555
finance_exp                            555
human_capital_exp                      555
consulting_exp                         555
founder_exp                            555
foundation_year                        555
years_of_experience_as_entreprenuer    555
age_of_first_entrepreneurship          555
age                                    555
years_of_experience                    555
number_of_startups                     555
gender                                 555
foreing_studies                        555
foreing_work_experience                555
foreing_founder                        555
dtype: int64

In [ ]:
# Encoding labels
le = LabelEncoder()
founders_df_Encode = founders_df.copy()
founders_df_Encode['general_location'] = le.fit_transform(founders_df_Encode['general_location'])
founders_df_Encode['gender'] = le.fit_transform(founders_df_Encode['gender'])

# Get the name of the columns
columnsName = founders_df_Encode.columns.tolist()

# convert variables to numerical
for column in columnsName:
  founders_df_Encode[column] = pd.to_numeric(founders_df_Encode[column], errors='coerce').astype('Int64').fillna(0)

founders_df_Encode.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,22,1,1,0,0,0,0,0,1,2020,12,31,32,13,4,1,0,0,0
1,2,23,22,0,1,0,0,0,0,0,1,2014,4,25,32,11,2,1,0,0,0
2,3,141,76,0,1,0,0,0,0,0,0,2015,0,22,28,6,1,1,0,1,0
3,4,152,22,0,0,0,0,1,0,1,1,2017,6,28,32,10,3,1,1,1,1
4,5,155,76,1,1,0,0,0,0,0,0,2015,10,27,33,16,1,1,1,0,0


In [ ]:
# data types
founders_df_Encode.dtypes

id_founder                             Int64
id_company                             Int64
general_location                       Int64
manager_exp                            Int64
clevel_exp                             Int64
engineering_exp                        Int64
sales_exp                              Int64
finance_exp                            Int64
human_capital_exp                      Int64
consulting_exp                         Int64
founder_exp                            Int64
foundation_year                        Int64
years_of_experience_as_entreprenuer    Int64
age_of_first_entrepreneurship          Int64
age                                    Int64
years_of_experience                    Int64
number_of_startups                     Int64
gender                                 Int64
foreing_studies                        Int64
foreing_work_experience                Int64
foreing_founder                        Int64
dtype: object

## Experience


In order to understand the founder's experience, we're going to calculate the percentage of founders who has the experience in each topic

In [ ]:
# Experience percentages
manager_exp_percentage = founders_df_Encode["manager_exp"].value_counts(1)[1]
clevel_exp_percentage = founders_df_Encode["clevel_exp"].value_counts(1)[1]
engineering_exp_percentage = founders_df_Encode["engineering_exp"].value_counts(1)[1]
sales_exp_percentage = founders_df_Encode["sales_exp"].value_counts(1)[1]
finance_exp_percentage = founders_df_Encode["finance_exp"].value_counts(1)[1]
human_capital_exp_percentage = founders_df_Encode["human_capital_exp"].value_counts(1)[1]
consulting_exp_percentage = founders_df_Encode["consulting_exp"].value_counts(1)[1]
founder_exp_percentage = founders_df_Encode["founder_exp"].value_counts(1)[1]


In [ ]:
# experience vector
# f"Winning Percentage: {winning_percentage:.1f}%\n"
experience = {
    "Management": f"{100*manager_exp_percentage:.1f}%",
    "C-Level": f"{100*clevel_exp_percentage:.1f}%",
    "Engineering": f"{100*engineering_exp_percentage:.1f}%",
    "Sales": f"{100*sales_exp_percentage:.1f}%",
    "Finance": f"{100*finance_exp_percentage:.1f}%",
    "Human Resources": f"{100*human_capital_exp_percentage:.1f}%",
    "Consulting": f"{100*consulting_exp_percentage:.1f}%",
    "Founder": f"{100*founder_exp_percentage:.1f}%",
}
experience_df = pd.DataFrame([experience])
experience_df

,Management,C-Level,Engineering,Sales,Finance,Human Resources,Consulting,Founder
0,49.5%,65.4%,18.6%,9.9%,10.1%,2.9%,21.6%,77.7%


**Some findindgs**
* Almost 8 out of 10 founders have founded other companies before
* 6 out of 10 have C-Level experience
* Half of founders have experience in management

## Entrepreneurship experience

It's important to know more about year of foundation, entrepreneurship experience, etc.

In [ ]:
# Year of foundation
year_foundation = founders_df_Encode["foundation_year"].value_counts()
year_foundation


2015    84
2016    63
2014    60
2019    53
2018    53
2017    52
2020    45
2013    44
2012    35
2011    19
2021    18
2010     9
2009     4
2008     4
2007     2
2003     2
2002     2
1999     2
2001     1
2006     1
2004     1
1997     1
Name: foundation_year, dtype: Int64

In [ ]:
# Maxim of companies in year_foundation
year_foundation_max = founders_df_Encode["foundation_year"].value_counts().max()
year_foundation_max


84

In [ ]:
# Years as entrepreneur
founders_df_Encode["years_of_experience_as_entreprenuer"].mean()

8.281081081081082

In [ ]:
# Age of first entrepreneurship
founders_df_Encode["age_of_first_entrepreneurship"].mean()

30.951351351351352

In [ ]:
# Founder's age
founders_df_Encode["age"].mean()

36.387387387387385

In [ ]:
# Years of overall experience
founders_df_Encode["years_of_experience"].mean()

13.601801801801802

In [ ]:
# Number of startups
founders_df_Encode["number_of_startups"].mean()

1.6846846846846846

**Some findings**
* Most of the companies were founded in 2015 (84) so, the average of experience of the companies is around 6 years
* The average of experience as entrepreneur is 8.3
* The average of age of first entrepreneurship es 31 years
* The average of founder's age is 36 years 
* The average of overall experience is 13.6 years
* The number of start ups es 1.7 in average


# **Companies**


## Data & cleaning

Data is already clean, because is the result of the union and manipulation of Founders' table and Rounds' table, which data is also clean 

In [ ]:
# Query to connect with SQL data base
SQL_Query = pd.read_sql_query('''select * FROM "companies" ''', conn)
companies_df = pd.DataFrame(SQL_Query)
companies_df










,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
0,3.0,2017.0,0.0,2017.0,1400000.0,2017.0,1400000.0,48.0,33.0,8.0,3.0,1.0,4.0,0.0,0.0,0.0,2017.0,1.0,1400000.0,1400000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,0.0,12.0,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,0.0,22.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,7.0
1,7.0,2018.0,0.0,2018.0,10500.0,2018.0,10500.0,36.0,10.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,10500.0,10500.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
2,10.0,2015.0,0.0,2015.0,200000.0,2015.0,200000.0,72.0,11.0,14.0,1.0,1.0,6.0,0.0,0.0,0.0,2015.0,1.0,200000.0,200000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
3,23.0,2013.0,0.0,2013.0,150000.0,2021.0,40000000.0,0.0,1001.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2021.0,5.0,54800000.0,40000000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
4,51.0,2018.0,0.0,2018.0,20000.0,2018.0,20000.0,36.0,10.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,20000.0,20000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,4429.0,2018.0,0.0,2018.0,750000.0,2021.0,3100000.0,0.0,259.0,249.0,0.0,1.0,0.0,0.0,1.0,1.0,2021.0,4.0,12510000.0,4960000.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,12.0,11.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,12.0,11.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,12.0,11.0
506,4433.0,2021.0,0.0,2021.0,225000000.0,2021.0,225000000.0,0.0,125.0,124.0,2.0,1.0,0.0,1.0,1.0,1.0,2021.0,1.0,225000000.0,225000000.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,9.0,8.0
507,4434.0,2021.0,0.0,2021.0,130000.0,2021.0,130000.0,0.0,11.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2021.0,1.0,130000.0,130000.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,6.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,6.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,6.0,4.0
508,4435.0,2021.0,0.0,2021.0,130000.0

In [ ]:
companies_df.dtypes

id_company                                     float64
foundation_year                                float64
category                                       float64
seed_year                                      float64
seed_amount                                    float64
                                                ...   
mean_foreing_studies                           float64
mean_foreing_work_experience                   float64
mean_foreing_founder                           float64
mean_dp_years_of_experience                    float64
mean_dp_years_of_experience_as_entreprenuer    float64
Length: 65, dtype: object

In [ ]:
companies_df.isna().sum()

id_company                                     0
foundation_year                                0
category                                       0
seed_year                                      0
seed_amount                                    0
                                              ..
mean_foreing_studies                           0
mean_foreing_work_experience                   0
mean_foreing_founder                           0
mean_dp_years_of_experience                    0
mean_dp_years_of_experience_as_entreprenuer    0
Length: 65, dtype: int64

In [ ]:
companies_df.count()

id_company                                     510
foundation_year                                510
category                                       510
seed_year                                      510
seed_amount                                    510
                                              ... 
mean_foreing_studies                           510
mean_foreing_work_experience                   510
mean_foreing_founder                           510
mean_dp_years_of_experience                    510
mean_dp_years_of_experience_as_entreprenuer    510
Length: 65, dtype: int64

In [ ]:
# Drop the null columns where all values are null
companies_df = companies_df.dropna(axis='columns', how='all')
companies_df.head()

,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
0,3.0,2017.0,0.0,2017.0,1400000.0,2017.0,1400000.0,48.0,33.0,8.0,3.0,1.0,4.0,0.0,0.0,0.0,2017.0,1.0,1400000.0,1400000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,0.0,12.0,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,0.0,22.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,7.0
1,7.0,2018.0,0.0,2018.0,10500.0,2018.0,10500.0,36.0,10.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,10500.0,10500.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
2,10.0,2015.0,0.0,2015.0,200000.0,2015.0,200000.0,72.0,11.0,14.0,1.0,1.0,6.0,0.0,0.0,0.0,2015.0,1.0,200000.0,200000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
3,23.0,2013.0,0.0,2013.0,150000.0,2021.0,40000000.0,0.0,1001.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2021.0,5.0,54800000.0,40000000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
4,51.0,2018.0,0.0,2018.0,20000.0,2018.0,20000.0,36.0,10.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,20000.0,20000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0


In [ ]:
import re
# Encoding labels
le = LabelEncoder()
db_companies_Encode = companies_df.copy()

# Get the name of the columns
columnsName = db_companies_Encode.columns.tolist()
columnsName.remove('rounds_funding_factor')
r = re.compile("^mean")
columnsName = [column for column in columnsName if not r.match(column)]

# convert variables to numerical
for column in columnsName:
  db_companies_Encode[column] = pd.to_numeric(db_companies_Encode[column], errors='coerce').fillna(0).astype(np.int64)


db_companies_Encode.head()

,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
0,3,2017,0,2017,1400000,2017,1400000,48,33,8,3,1,4.0,0,0,0,2017,1,1400000,1400000,1,1,0,0,0,0,0,1,3,1,1,0,0,12,8,1,1,0,0,0,0,0,2,4,2,1,0,0,22,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,7.0
1,7,2018,0,2018,10500,2018,10500,36,10,0,2,0,3.0,0,0,0,2018,1,10500,10500,1,1,0,0,0,0,0,1,4,1,0,0,0,10,9,1,1,0,0,0,0,0,1,4,1,0,0,0,10,9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
2,10,2015,0,2015,200000,2015,200000,72,11,14,1,1,6.0,0,0,0,2015,1,200000,200000,1,1,0,0,0,0,0,1,1,1,0,0,0,12,2,1,1,0,0,0,0,0,1,1,1,0,0,0,12,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
3,23,2013,0,2013,150000,2021,40000000,0,1001,0,0,1,0.0,1,1,1,2021,5,54800000,40000000,0,1,0,0,0,0,0,1,2,1,0,0,0,11,4,0,1,0,0,0,0,0,1,2,1,0,0,0,11,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
4,51,2018,0,2018,20000,2018,20000,36,10,0,1,0,3.0,0,0,0,2018,1,20000,20000,0,1,1,0,0,0,0,0,1,1,1,0,0,5,-1,0,1,1,0,0,0,0,0,1,1,1,0,0,5,-1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0


In [ ]:
# data types
db_companies_Encode.dtypes

id_company                                       int64
foundation_year                                  int64
category                                         int64
seed_year                                        int64
seed_amount                                      int64
                                                ...   
mean_foreing_studies                           float64
mean_foreing_work_experience                   float64
mean_foreing_founder                           float64
mean_dp_years_of_experience                    float64
mean_dp_years_of_experience_as_entreprenuer    float64
Length: 65, dtype: object

In [ ]:
db_companies_Encode.isna().sum()

id_company                                     0
foundation_year                                0
category                                       0
seed_year                                      0
seed_amount                                    0
                                              ..
mean_foreing_studies                           0
mean_foreing_work_experience                   0
mean_foreing_founder                           0
mean_dp_years_of_experience                    0
mean_dp_years_of_experience_as_entreprenuer    0
Length: 65, dtype: int64

In [ ]:
db_companies_Encode.count()

id_company                                     510
foundation_year                                510
category                                       510
seed_year                                      510
seed_amount                                    510
                                              ... 
mean_foreing_studies                           510
mean_foreing_work_experience                   510
mean_foreing_founder                           510
mean_dp_years_of_experience                    510
mean_dp_years_of_experience_as_entreprenuer    510
Length: 65, dtype: int64

## Exploration

In [ ]:
# Foundation year
db_companies_Encode["foundation_year"].value_counts()

2016    73
2015    66
2017    62
2014    57
2013    49
2019    47
2020    43
2018    40
2021    30
2012    24
2011    10
2008     3
2010     2
2009     1
2006     1
2004     1
2000     1
Name: foundation_year, dtype: int64

In [ ]:
# Seed year
db_companies_Encode["seed_year"].value_counts()

2016    73
2015    66
2017    62
2014    57
2013    49
2019    47
2020    43
2018    40
2021    30
2012    24
2011    10
2008     3
2010     2
2009     1
2006     1
2004     1
2000     1
Name: seed_year, dtype: int64

In [ ]:
# Seed amount
seed_amount_mean = db_companies_Encode["seed_amount"].mean()
seed_amount_max = db_companies_Encode["seed_amount"].max()
seed_amount_min = db_companies_Encode["seed_amount"].min()
print(seed_amount_mean)
print(seed_amount_max)
print(seed_amount_min)

3643517.0803921567
500000000
0


In [ ]:
# Last round amount
amount_last_mean = db_companies_Encode["amount_last_funding"].mean()
amount_last_max = db_companies_Encode["amount_last_funding"].max()
amount_last_min = db_companies_Encode["amount_last_funding"].min()
print(amount_last_mean)
print(amount_last_max)
print(amount_last_min)

7959460.270588235
700000000
1000


In [ ]:
# Time passed since last funding

db_companies_Encode["month_since_last_funding"].mean()

44.56470588235294

In [ ]:
# Number of employees
print(db_companies_Encode["month_since_last_funding"].mean())
print(db_companies_Encode["increase_employees"].mean())

44.56470588235294
35.64313725490196


In [ ]:
# Current status
db_companies_Encode["current_status"].value_counts(1)[1]

0.7725490196078432

In [ ]:
# Soonicorn
db_companies_Encode["soonicorn"].value_counts(1)[1]

0.08235294117647059

In [ ]:
# More than 50 employees
db_companies_Encode["more50_employees"].value_counts(1)[1]

0.23333333333333334

In [ ]:
# More than 100 employees
db_companies_Encode["more100_employees"].value_counts(1)[1]

0.14313725490196078

In [ ]:
# Rounding factor 
db_companies_Encode["rounds_funding_factor"].mean()

2.537254901960784

**Some findings**
This data will be better explained when the model could be able to separate between sucessful and not sucessful companies
* The foundations years most popular are 2015 and 2016, the very same years of the first round of funding
* The avarage amount of seed round is 364,517.08, the range goes from 0.00 to 500,000,000.00 (to many zeros!)
* The average amount of last round is 7,959,460.27, the range is higher than the seed round, from 1,000.00 to 700,000,000.00
* In general, there are 44 months since the last funding round
* The average of employees in this companies is 44.6, however, this number is increased by 35 employees at the moment
* 77% of these companies are Actives at the moment of this analysis
* Less than 10% of the companies are Soonicorn (42)
* 23% of the companies have already more than 50 employees and just the 14% have rolled over 100
* The average round factor is 2.54


